In [7]:
import cv2  
import math 

from ultralytics import YOLO

NameError: name '_C' is not defined

In [5]:
import os
import random
import shutil

# Carpeta base del notebook
base_dir = os.getcwd()  # suponiendo que estás en Práctica_4/Parte_A

# Carpeta donde están tus imágenes y labels
source_images = os.path.join(base_dir, "images", "Matrículas")
source_labels = os.path.join(source_images, "labels")

# Listar imágenes
images = [f for f in os.listdir(source_images) if f.endswith((".jpg", ".png"))]

# Mezclar aleatoriamente
random.shuffle(images)

# Porcentajes de división
train_split = 0.7
val_split = 0.2

num_total = len(images)
num_train = int(num_total * train_split)
num_val = int(num_total * val_split)
num_test = num_total - num_train - num_val

# Carpeta destino
dest_base = os.path.join(base_dir, "TGC_RBNW")

# Crear carpetas si no existen
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(dest_base, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(dest_base, split, "labels"), exist_ok=True)

# Función para mover imagen y su label
def move_files(split_name, files):
    for img_file in files:
        label_file = img_file.rsplit(".", 1)[0] + ".txt"
        # mover imagen
        shutil.move(os.path.join(source_images, img_file), os.path.join(dest_base, split_name, "images", img_file))
        # mover label solo si existe
        label_path = os.path.join(source_labels, label_file)
        if os.path.exists(label_path):
            shutil.move(label_path, os.path.join(dest_base, split_name, "labels", label_file))
            print(f"Encontrado {img_file}, no se omite.")

        else:
            print(f"⚠️ Label no encontrado para {img_file}, se omite.")

# Repartir
move_files("train", images[:num_train])
move_files("val", images[num_train:num_train + num_val])
move_files("test", images[num_train + num_val:])


In [ ]:
data_path = r"C:\Users\luisp\Desktop\VC\prac1\P4\matriculas\data\matriculas.yaml"
model = YOLO("yolo11n.pt")

results = model.train(
    data=data_path,
    epochs=120,             # más largo + patience
    patience=30,
    imgsz=1280,             # sube resolución (mejora detalle)
    batch=-1,               # AutoBatch
    device=0,
    amp=True,
    rect=True,              # mantiene aspect ratio (mejor para cajas finas)
    workers=4,

    optimizer="adamw",
    cos_lr=True,
    lr0=8e-4,               # un poco más bajo para afinar
    lrf=1e-2,

    # Aumentos centrados en “placa pequeña / vídeo”
    mosaic=0.25,            # menos agresivo → mejores cajas
    close_mosaic=15,
    mixup=0.0,              # fuera (deforma letras)
    copy_paste=0.0,
    fliplr=0.0,             # no inviertas texto
    degrees=3.0,            # leve tilt
    translate=0.05,
    scale=0.50,             # importante: genera placas muy pequeñas y grandes
    shear=0.0,
    perspective=0.0,
    hsv_h=0.015, hsv_s=0.4, hsv_v=0.4,  # cambios de luz contrastes

    # Guardados y nombre
    save_period=10,
    project="runs/detect",
    name="plates_s_1280_rect",
    exist_ok=True,
)

In [ ]:
import cv2
import math
import csv
from ultralytics import YOLO
from collections import defaultdict

# Modelo entrenado
model = YOLO("runs/detect/train/weights/best.pt")

# Video de entrada
filename = "video_test.mp4"
vid = cv2.VideoCapture(filename)

# Diccionario para guardar historial de tracking
track_history = defaultdict(list)

# CSV
csv_file = open("detecciones.csv", "w", newline="")
csv_writer = csv.writer(csv_file)
csv_writer.writerow([
    "fotograma", "tipo_objeto", "confianza", "id_tracking", 
    "x1","y1","x2","y2",
    "matricula_en_su_caso", "conf_matricula", "mx1","my1","mx2","my2","texto_matricula"
])

frame_num = 0
while True:
    ret, img = vid.read()
    if not ret:
        break

    results = model.track(img, persist=True, classes=[0,2])  # tracking solo personas (0) y vehículos (2)

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = [int(coord) for coord in box.xyxy[0]]
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            track_id = int(box.id[0]) if box.id is not None else -1

            # Aquí puedes integrar OCR para matrícula si cls==2 (vehículo)
            matricula = ""
            conf_matricula = 0.0
            mx1 = my1 = mx2 = my2 = 0

            csv_writer.writerow([
                frame_num, cls, conf, track_id,
                x1,y1,x2,y2,
                matricula, conf_matricula, mx1,my1,mx2,my2, matricula
            ])

            # Dibuja la caja en el vídeo
            color = (0,255,0) if cls==0 else (0,0,255)
            cv2.rectangle(img, (x1,y1), (x2,y2), color, 2)
            cv2.putText(img, f"{cls}-{track_id}", (x1,y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Deteccion y Tracking", img)
    frame_num += 1
    if cv2.waitKey(1) & 0xFF == 27:
        break

vid.release()
cv2.destroyAllWindows()
csv_file.close()
